# The Tavern alcohol sales

A quick look at alcohol sales by The Tavern, as reported to the Texas Comptroler.

The long-time Austin institution has been [put up for sale by its owners](http://www.austin360.com/entertainment/longtime-austin-bar-the-tavern-now-for-sale-but-might-not-change-much/0Nwy3ifTfq0mzn5NOyiKYJ/).

The data was downloaded from [data.texas.gov](https://data.texas.gov/Government-and-Taxes/Mixed-Beverage-Gross-Receipts/naix-2893), filtering on the `location_address` of "922 W 12TH ST".

This data was pulled on 12/22/2018 for this story, and receipts for 2017 were only up to date through October 2017.

This notebook should be fully repeatable by downloading this repo, then using a python virtual environment (requirements.txt included) to run the code using [Jupyter](http://jupyter.org/).

In [1]:
%%bash
# download the file
curl -L -o ../data-orig/the-tavern.csv \
https://data.texas.gov/resource/fp9t-htqh.csv?Location%20Address=922%20W%2012TH%20ST

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 34891    0 34891    0     0  96428      0 --:--:-- --:--:-- --:--:-- 96383


In [2]:
import agate

In [3]:
# This is the source file that was just downloaded
file = '../data-orig/the-tavern.csv'

In [4]:
#Helps us import some text fields that may be considered numbers in error.
specified_types = {
    'cover_charge_receipts': agate.Number(),
    'taxpayer_number': agate.Text(),
    'taxpayer_zip': agate.Text(),
    'taxpayer_county': agate.Text(),
    'location_number': agate.Text(),
    'location_address': agate.Text(),
    'location_zip': agate.Text(),
    'location_county': agate.Text(),
}

# this imports the file specified above, along with the proper types
data = agate.Table.from_csv(file, column_types=specified_types)

# renames two columns that are labelled incorrectly from original data
tavern = data.rename(column_names = {
        'tabc_permit_number': 'inside_outside_city_limits_code',
        'inside_outside_city_limits_code_y_n': 'tabc_permit',
    }
)

# prints table fields so we an check thoes data types
print(tavern)
print('Number of records: {}'.format(len(tavern)))

| column                             | data_type |
| ---------------------------------- | --------- |
| beer_receipts                      | Number    |
| cover_charge_receipts              | Number    |
| tabc_permit                        | Text      |
| liquor_receipts                    | Number    |
| location_address                   | Text      |
| location_city                      | Text      |
| location_county                    | Text      |
| location_name                      | Text      |
| location_number                    | Text      |
| location_state                     | Text      |
| location_zip                       | Text      |
| obligation_end_date_yyyymmdd       | DateTime  |
| responsibility_begin_date_yyyymmdd | DateTime  |
| responsibility_end_date_yyyymmdd   | DateTime  |
| inside_outside_city_limits_code    | Boolean   |
| taxpayer_address                   | Text      |
| taxpayer_city                      | Text      |
| taxpayer_county              

In [5]:
# showing there is only on location_address
print('All permutations of the address in data:')
print(tavern.columns['location_address'].values_distinct())
print('All permutations of the name in data:')
print(tavern.columns['location_name'].values_distinct())

All permutations of the address in data:
('922 W 12TH ST',)
All permutations of the name in data:
('THE TAVERN', 'TAVERN')


In [6]:
# create some date-related columns for grouping
tavern = tavern.compute([
        ('year', agate.Formula(agate.Text(), lambda r: r['obligation_end_date_yyyymmdd'].year)),
        ('month', agate.Formula(agate.Text(), lambda r: r['obligation_end_date_yyyymmdd'].month))
    ], replace=True)

In [7]:
# Group by year so we can sum them
tavern_year = tavern.group_by('year')

# Create an aggregation of Total Receipts
tavern_year_sums = tavern_year.aggregate([
        ('total', agate.Sum('total_receipts'))
    ])

# ordering rows for pretty chart
tavern_year_ordered = tavern_year_sums.order_by('year')


In [8]:
# print chart
tavern_year_ordered.print_bars('year', 'total', width=80)

year   total
2007 945,796 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░    
2008 859,701 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░         
2009 706,290 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░                   
2010 776,630 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░               
2011 881,739 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░        
2012 960,378 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░   
2013 874,131 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░        
2014 841,802 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░          
2015 734,081 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░                  
2016 622,976 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░                         
2017 487,529 ▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░                                  
             +---------------+----------------+----------------+---------------+
             0 

## Conclusion
At the time this report, 2016 was the lowest full year in total alcohol receipts reported to the Comptroller since 2007.